# Monte Carlo - Unidad 2, Sesion 3 - Ejercicio

**[Problema]**: se desea estimar el volumen de una region R de [0,1]^6 definida por todos los puntos de la hiper-esfera de centro (0.45, 0.5, 0.6, 0.6, 0.5, 0.45) y radio 0.35 que ademas cumplen con las restricciones 3x1 + 7x4 <= 5; x3+x4 <= 1; x1-x2-x5+x6 >= 0

## Entrega 2 - Ejercicio 3.1

### Parte a: 

**[Letra]** Implementar un programa que reciba como parametro la
cantidad de replicaciones n a realizar, y emplee Monte Carlo para
calcular (e imprimir) la estimaci´on del volumen de R, y la desviaci´on
estandar de este estimador. Incluir codigo para calcular el tiempo de
calculo empleado por el programa. Utilizar el programa con n = 104 y
luego con n = 106 para estimar el volumen de R. Discutir si los dos
valores obtenidos parecen consistentes.
(en la sesi´on 5 se continuar´a este ejercicio).

In [3]:
import random
import math
import tabulate
import time
random.seed()

def sortearPuntoRN(dim=2):
    """
    Seortea un punto en R^N dentro del hiper-cubo [0,1]^N
    """
    punto = []
    for n in range(0, dim):
        punto.append(random.uniform(0.0, 1.0))
    # end for

    return punto
# end fun sortearPuntoRN

def puntoDentroVolumen(punto, restricciones=True):
    """
    Devuelve 0 o 1 si un punto esta fuera o dentro de un cierto volumen.
    Si restricciones es "false", el volumen es la hiperesfera en R6
    """

    # Para que este dentro del volumen tiene que estar dentro de la esfera 
    # y ademas cumplir con las restricciones adicionales

    dentro = 1
    fuera = 0

    # chequeo 1 : dentro de esfera

    d = math.sqrt(
        (punto[0]-0.45)**2 +
        (punto[1]-0.5)**2 +
        (punto[2]-0.6)**2 +
        (punto[3]-0.6)**2 +
        (punto[4]-0.5)**2 +
        (punto[5]-0.45)**2 
    )

    # si la distancia es mayor al radio, esta fuera
    if (d>=0.35) : 
        return fuera

    if restricciones:
        # restriccion 1
        if 3*punto[0] + 7*punto[3] > 5:
            return fuera
        # restriccion 2
        if punto[2]+punto[3] > 1:
            return fuera
        # restriccion 3
        if punto[0]-punto[1]-punto[4]+punto[5] < 0:
            return fuera
    else:
        return dentro

    return dentro
# end fun punto dentro del volumen

# sortearPuntoRN(6)

In [4]:
# Implemento pseudocodigo Montecarlo

def MetodoMonteCarlo(N, FVolumen):
    """
    Implementa el pseudocodigo de MC
    N: cantidad de muestras
    FVolumen: funcion que define el volumen, devuelve 0 si el punto esta fuera, 1 si esta dentro
    """
    t0 = time.perf_counter()
    S = 0
    for j in range(0, N):
        punto = sortearPuntoRN(6)
        if FVolumen(punto):
            phi = 1
        else: 
            phi = 0
        S = S + phi
    # end for
    VolR = S / N
    VarVorR = (S/N)*(1-S/N)/(N-1)
    return (VolR, VarVorR, time.perf_counter()-t0)
# end def

VolH = math.pi**3*(0.35**6)/6

(VolR, VarVolR, execTime) = MetodoMonteCarlo(10**6, lambda x: puntoDentroVolumen(x, False))



### Verificacion

Comparamos el volumen sin restricciones con el volumen calculado analiticamente de la hiperesfera en R6

In [5]:
print("Volumen hiper esfera por MMC = {:e}, Varianza = {:e}".format(VolR, VarVolR))
print(" ")
print("Volumen hiper esfera analitico = {:e}, diferencia MMC - analitico = {:.3f}%".format(VolH, (VolH-VolR)/VolR*100))

Volumen hiper esfera por MMC = 9.596000e-03, Varianza = 9.503926e-09
 
Volumen hiper esfera analitico = 9.499629e-03, diferencia MMC - analitico = -1.004%


Con un millon de muestras tenemos una diferencia de menos de 1% entre el volumen calculado de forma analitica y el volumen calculado por Montecarlo.

### Ejecucion para diferentes tamanos de muestra

En esta seccion corremos MMC para calcular el volumen con restricciones para diferentes tamanos de muestra.

In [6]:
table = [ ['N', 'Vol hiperesfera (analitico)', 'Vol hiperesfera+restricciones', 'Varianza', 'Tiempo (s)'] ]

for n in [2, 3, 4, 5, 6]:
    (VolR, VarVolR, execTime) = MetodoMonteCarlo(10**n, lambda x: puntoDentroVolumen(x, True))
    table.append( [10**n, "{:3e}".format(VolH), "{:3e}".format(VolR), "{:3e}".format(VarVolR), "{:3f}".format(execTime)] )


tabulate.tabulate(table, tablefmt='html')

N,Vol hiperesfera (analitico),Vol hiperesfera+restricciones,Varianza,Tiempo (s)
100,9.499629e-03,0.000000e+00,0.000000e+00,0.000736
1000,9.499629e-03,0.000000e+00,0.000000e+00,0.014217
10000,9.499629e-03,4.000000e-04,3.998800e-08,0.109213
100000,9.499629e-03,2.800000e-04,2.799244e-09,0.486946
1000000,9.499629e-03,2.900000e-04,2.899162e-10,4.810353


Entre las corridas de 10mil y 1millon de muestras hay una diferencia de un 7.6% aproximadamente. Los resultados parecen coherentes en el sentido de que al aumentar el tamano de la muestra el resultado parece tender a un valor y no parece diverger. La varianza estimada tambien decrece al aumentar el tamano de la muestra, otro resultado esperable.

El volumen determinado para la hiperesfera con restricciones es consistentemente menor al volumen de la hiperesfera sin restricciones, lo cual tiene sentido ya que las restricciones justamente eliminan puntos del volumen en cuestión.

## Entrega 3 : Ejercicio 5.1

Para el mismo enunciado de mas arriba (estimación de un volumen con restricciones) se pide:

## Parte a

**[Letra]**: Compartir en el grupo los códigos desarrollados para la parte a,
validarlos revisando los códigos, y verificando si las salidas para tamaños
de muestra de 106 son consistentes. Indicar si se detectaron errores en
los mismos, y en ese caso dar los códigos corregidos. Elegir uno de los
códigos para las partes siguientes, explicar los motivos de la selecci´on.

*Por el momento sigo trabajando cno mi código en Python ya que llegué con retraso a la elección de grupo*

## Parte b

**[Letra]**: calcular la cantidad de replicaciones a realizar para garantizar
un error menor a 1:0 × 10−4 con probabilidad 0:95, utilizando el criterio
de peor caso de Hoeffding

In [13]:
# Formula de Hoeffding

def tamMuestraHoeffding(epsilon, delta):
    num = 2 * math.log(2/delta)
    den = 4 * epsilon**2
    return math.ceil(num/den)
# end def

f'{tamMuestraHoeffding(0.0001, 0.05):,}'

'184,443,973'